### **What is Neural Network**
Neural network is a collection of neurons that are connected by layers. Each neuron is a small computing unit that performs simple calculations to collectively solve a problem. They are organized in layers. There are 3 types of layers: input layer, hidden layer and outter layer. Each layer contains a number of neurons, except for the input layer. Neural networks mimic the way a human brain processes information.

## **Components of a neural network**

- **Activation function** determines whether a neuron should be activated or not. The computations that happen in a neural network include applying an activation function. If a neuron activates, then it means the input is important.  The are different kinds of activation functions. The choice of which activation function to use depends on what you want the output to be. Another important role of an activation function is to add non-linearity to the model.
    - _Binary_ used to set an output node to 1 if function result is positive and 0 if the function result is negative.  f(x)= \begin{cases} 0, & \text{if } x < 0\\ 1, & \text{if } x\geq 0\\ \end{cases}
    - _Sigmod_ is used to predict the probability of an output node being between 0 and 1.  $f(x) = {\large \frac{1}{1+e^{-x}}} $
    - _Tanh_ is used to predict if an output node is between 1 and -1.  Used in classification use cases.  $f(x) = {\large \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}} $
    - _ReLU_ used to set the output node to 0 if fuction result is negative and keeps the result value if the result is a positive value.  f(x)= \begin{cases} 0, & \text{if } x < 0\\ x, & \text{if } x\geq 0\\ \end{cases}
- **Weights** influence how well the output of our network will come close to the expected output value. As an input enters the neuron, it gets multiplied by a weight value and the resulting output is either observed, or passed to the next layer in the neural network. Weights for all neurons in a layer are organized into one tensor
- **Bias** makes up the difference between the activation function's output and its intended output. A low bias suggest that the network is making more assumptions about the form of the output, whereas a high bias value makes less assumptions about the form of the output. 



We can say that an output $y$ of a neural network layer with weights $W$ and bias $b$ is computed as summation of the inputs multiply by the weights plus the bias $x = \sum{(weights * inputs) + bias} $, where $f(x)$ is the activation function.

### **Build a Neural Network**
Neural networks are comprised of layers/modules that perform operations on data. The ***torch.nn*** namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the ***nn.Module***. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the ***FashionMNIST*** dataset.



In [1]:
%matplotlib inline
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### **Get a hardware device for training**
We want to be able to train our model on a hardware accelerator like the GPU, if it is available. Let's check to see if ***torch.cuda*** is available, else we continue to use the CPU.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


### **Define the class**
We define our neural network by subclassing `nn.Module`, and 
initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements
the operations on input data in the `forward` method.

Our neural network are composed of the following:
- The input layer with 28x28 or 784 features/pixels.
- The first linear module takes the input 784 features and transforms it to a hidden layer with 512 features
- The ReLU activation function will be applied in the transformation
- The second linear module take 512 features as input from the first hidden layer and transforms it to the next hidden layer with 512 features
- The ReLU activation function will be applied in the transformation
- The third linear module take 512 features as input from the second hidden layer and transforms it to the output layer with 10, which is the number of classes
- The ReLU activation function will be applied in the transformation

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print 
it's structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model's `forward`, along with some background operations. However, do not call `model.forward()` directly! Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.

We get the prediction densities by passing it through an instance of the `nn.Softmax`.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


### **Weight and Bias**


The `nn.Linear` module randomly initializes the ${weights}$  and ${ bias}$ for each layer and internally stores the values in Tensors.

In [7]:
print(f"First Linear weights: {model.linear_relu_stack[0].weight} \n")
print(f"First Linear bias: {model.linear_relu_stack[0].bias} \n")

print(f"Second Linear weights: {model.linear_relu_stack[2].weight} \n")
print(f"Second Linear bias: {model.linear_relu_stack[2].bias} \n")

print(f"Third Linear weights: {model.linear_relu_stack[4].weight} \n")
print(f"Third Linear bias: {model.linear_relu_stack[4].bias} \n")

First Linear weights: Parameter containing:
tensor([[-0.0177,  0.0154,  0.0266,  ..., -0.0189,  0.0246,  0.0332],
        [ 0.0222,  0.0013,  0.0174,  ..., -0.0315, -0.0180, -0.0105],
        [ 0.0344, -0.0357,  0.0262,  ..., -0.0310, -0.0094, -0.0159],
        ...,
        [ 0.0089, -0.0090, -0.0161,  ..., -0.0065,  0.0018, -0.0265],
        [ 0.0079,  0.0319, -0.0139,  ..., -0.0118,  0.0138,  0.0230],
        [-0.0079,  0.0316, -0.0274,  ..., -0.0235, -0.0238,  0.0110]],
       device='cuda:0', requires_grad=True) 

First Linear bias: Parameter containing:
tensor([-9.3269e-03, -5.0084e-03,  2.6997e-02,  6.5094e-03,  2.7231e-02,
        -2.0737e-02,  2.8385e-02,  1.8674e-02,  2.7268e-02,  3.0006e-02,
         7.4255e-03,  1.4636e-02, -1.2799e-02,  3.4605e-02,  2.6423e-02,
        -2.9142e-02,  6.8262e-03, -5.8449e-03, -2.8626e-02,  3.1918e-02,
         1.8669e-02,  2.5236e-02, -1.7747e-02,  6.2139e-03, -2.4140e-02,
         4.6706e-03, -1.1668e-02, -8.5742e-03, -3.1915e-03,  1.5785e-0

### **Model Layers**
Let's break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size **28x28** and see what happens to it as we pass it through the network.

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### **nn.Flatten()**
We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained). Each of the pixels are pass to the input layer of the neural network.



In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### **nn.Linear()**
The linear layer is a module that applies a linear transformation on the input using it's stored weights and biases.  The gayscale value of each pixel in the input layer will be connected to neurons in the hidden layer for calculation.    The calculation used for the transformation is ${{weight * input + bias}} $.


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### **nn.ReLu()**
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena. In this model, we use `nn.ReLU` between our linear layers, but there's other activations to introduce non-linearity in your model.

The ReLU activation function takes the output from the linear layer calculation and replaces the negative values with zeros.

Linear output: ${ x = {weight * input + bias}} $.  
ReLU:  $f(x)= 
\begin{cases}
    0, & \text{if } x < 0\\
    x, & \text{if } x\geq 0\\
\end{cases}
$

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.6866, -0.0722,  0.1892, -0.1151,  0.4069,  0.0607, -0.3388, -0.4251,
         -0.4247,  0.3258,  0.0516, -0.0228,  0.0859, -0.1371,  0.0395,  0.4166,
         -0.1565, -0.2604, -0.1388, -0.3579],
        [-0.3449,  0.1891, -0.4025,  0.1071,  0.0942, -0.0858, -0.4683, -0.5017,
         -0.4711, -0.1020, -0.0227,  0.0118,  0.1827, -0.1298,  0.0461,  0.4697,
         -0.2050, -0.2217, -0.3078, -0.3746],
        [-0.4174,  0.1864,  0.0421,  0.2093,  0.1181, -0.0829, -0.8770, -0.3323,
         -0.3541, -0.0798,  0.0895, -0.0758,  0.2901, -0.3273,  0.2835,  0.5054,
         -0.0410, -0.3443, -0.3308, -0.4529]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.1892, 0.0000, 0.4069, 0.0607, 0.0000, 0.0000, 0.0000,
         0.3258, 0.0516, 0.0000, 0.0859, 0.0000, 0.0395, 0.4166, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.1891, 0.0000, 0.1071, 0.0942, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0118, 0.1827, 0.0000, 0.04

### **nn.Sequential()**
`nn.Sequential` is an ordered 
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like `seq_modules`.

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### **nn.Softmax()**
The last linear layer of the neural network returns `logits` - raw values in \[`-infty`, `infty`], which are passed to the
`nn.Softmax` module. The Softmax activation function is used to calculate the probability of the output from the neural network.  It is only used on the output layer of a neural network.  The results are scaled to values \[0, 1\] representing the model's predicted densities for each class. `dim` parameter indicates the dimension along which the result values must sum to 1.  The node with the highest probability predicts the desired output.


In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### **Model Parameters**
Many layers inside a neural network are *parameterized*, i.e. have associated weights 
and biases that are optimized during training. Subclassing `nn.Module` automatically 
tracks all fields defined inside your model object, and makes all parameters 
accessible using your model's `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [14]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0177,  0.0154,  0.0266,  ..., -0.0189,  0.0246,  0.0332],
        [ 0.0222,  0.0013,  0.0174,  ..., -0.0315, -0.0180, -0.0105]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0093, -0.0050], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0159,  0.0430,  0.0168,  ..., -0.0392,  0.0252,  0.0038],
        [ 0.0234, -0.0262,  0.0440,  ...,  0.0391,  0.0355,  0.0173]],
       device='cu